# 必要なライブラリのimport

In [30]:
import requests
from selenium import webdriver
from webdriver_manager.utils import chrome_version
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import chromedriver_binary
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 

## accountのせってい

In [31]:
account_id = [アカウント情報を入力してください]
account_pass=[アカウントのパスワードを入力してください]

In [32]:
account_id_test = [アカウント情報を入力してください]
account_pass_test=[アカウントのパスワードを入力してください]

## userデータの取得

In [33]:
df = pd.read_excel("twitter_修正依頼.xlsx").ID

In [34]:
df = pd.read_csv("note_twitter0_4999.csv")["0"]

df = "https://twitter.com/" + df

In [35]:
df

0               https://twitter.com/issei_y
1                https://twitter.com/konpyu
2       https://twitter.com/shiraishikatsuy
3                https://twitter.com/shu223
4             https://twitter.com/snakajima
                       ...                 
4995     https://twitter.com/tane1_takafumi
4996                                  error
4997             https://twitter.com/A0314Z
4998                                  error
4999                                  error
Name: 0, Length: 5000, dtype: object

In [36]:
len(df)

5000

In [37]:
#200区切りに出来るように調整
df1 = df.loc[:8199]
df2 = df.loc[8200:]

In [38]:
df1.shape

(5000,)

In [39]:
df2.shape

(0,)

In [40]:
#~8200  shapeを(100, -1)に変更(100件ずつ処理できるようにするために)
box = []
box =  np.array(box)
box = box.reshape(-1,)
for i in range(1000):
    dff = np.array(df1[i*100: i*100+100])
    box = np.hstack((box,dff))
box = box.reshape(50,-1)
dfdf1 = pd.DataFrame(box.T)

In [41]:
#8200~
box = []
box =  np.array(box)
box = box.reshape(-1,)
for i in range(1):
    dff = np.array(df2[i*100: i*100+100])
    box = np.hstack((box,dff))
box = box.reshape(1,29)
dfdf2 = pd.DataFrame(box.T)

ValueError: cannot reshape array of size 0 into shape (1,29)

In [42]:
#8200~
box = []
box =  np.array(box)
box = box.reshape(-1,)
for i in range(1):
    dff = np.array(df[i*100: i*100+100])
    box = np.hstack((box,dff))
box = box.reshape(1,100)
dfdf2 = pd.DataFrame(box.T)

In [43]:
dfdf1.shape

(100, 50)

## twitterのフォロー、フォロワー、dm可否を確認できる関数の設定

In [44]:
class GetTwitter:
    def __init__(self):
        #seleniumのオプション
        self.options=Options()
        self.options.add_argument("no-sandbox")
        self.options.add_argument("--disable-extensions")
#         self.options.add_argument("--headless")
        self.options.add_argument('--disable-gpu')  
        self.options.add_argument('--ignore-certificate-errors')
        self.options.add_argument('--allow-running-insecure-content')
        self.options.add_argument('--disable-web-security')
        self.options.add_argument('--disable-desktop-notifications')
        self.options.add_argument("--disable-extensions")
        self.options.add_argument('--lang=ja')
        self.options.add_argument('--blink-settings=imagesEnabled=false')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--proxy-server="direct://"')
        self.options.add_argument('--proxy-bypass-list=*')    
        self.options.add_argument('--start-maximized')
        #twitterのログイン多面
        self.home_url="https://twitter.com/home"
        #最終的なリストを作成するためのせってい
        self.url_wrapper          = []
        self.follow_wrapper    = []
        self.follower_wrapper = []
        self.dm_wrapper          = []
        #エラーが起きたときにその時の番号をわかるようにする。
        self.set_error = []
     
    def login(self,tw_id,password):
        driver = webdriver.Chrome(chrome_options=self.options)
        driver.get(self.home_url)
        time.sleep(5)
        input_name=  driver.find_element_by_name("session[username_or_email]")
        input_name.send_keys(tw_id)

        #     パスワードのinput
        input_pass = driver.find_element_by_name("session[password]")
        input_pass.send_keys(password)

        #     ログインボタンのプッシュ
        login_button = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[2]/form/div/div[3]/div/div')
        login_button.click()

        time.sleep(1)
        return driver
    
    def get_tweet(self,urls,tw_id,password,instance,url_list,follow_list,follower_list,dm_list):
        driver=instance.login(tw_id,password)
        #urlリストを順々に取得
        print(len(urls),"回ループを起こします")
        for i,url in enumerate(urls):
            if  url != "error":
                try:
                    #順々にリストを巡回
                    driver.get(url)
                    time.sleep(1)
                    #driverのurlからhtmlを解析
                    html = driver.page_source.encode('utf-8')
                    soup = BeautifulSoup(html, 'lxml')
                    #鍵なしか鍵ありかでループを変える
                    
                    #そもそもとんだurlでifを回せるかどうか
                    try:
                        #鍵マークのhtml
                        key = soup.find_all("path",attrs={"d":"M19.75 7.31h-1.88c-.19-3.08-2.746-5.526-5.87-5.526S6.32 4.232 6.13 7.31H4.25C3.01 7.31 2 8.317 2 9.56v10.23c0 1.24 1.01 2.25 2.25 2.25h15.5c1.24 0 2.25-1.01 2.25-2.25V9.56c0-1.242-1.01-2.25-2.25-2.25zm-7 8.377v1.396c0 .414-.336.75-.75.75s-.75-.336-.75-.75v-1.396c-.764-.3-1.307-1.04-1.307-1.91 0-1.137.92-2.058 2.057-2.058 1.136 0 2.057.92 2.057 2.056 0 .87-.543 1.61-1.307 1.91zM7.648 7.31C7.838 5.06 9.705 3.284 12 3.284s4.163 1.777 4.352 4.023H7.648z"})
                        #鍵無し
                        if len(key) == 0:
                            #follow
                            follow  =driver.find_element_by_xpath("//a[contains(@href,'following')]").text.replace("Following","")
                            #follower
                            follower  =driver.find_element_by_xpath("//a[contains(@href,'follower')]").text.replace("Followers","")
                            #dm
                            dm = soup.find_all("path",attrs={"d":"M19.25 3.018H4.75C3.233 3.018 2 4.252 2 5.77v12.495c0 1.518 1.233 2.753 2.75 2.753h14.5c1.517 0 2.75-1.235 2.75-2.753V5.77c0-1.518-1.233-2.752-2.75-2.752zm-14.5 1.5h14.5c.69 0 1.25.56 1.25 1.25v.714l-8.05 5.367c-.273.18-.626.182-.9-.002L3.5 6.482v-.714c0-.69.56-1.25 1.25-1.25zm14.5 14.998H4.75c-.69 0-1.25-.56-1.25-1.25V8.24l7.24 4.83c.383.256.822.384 1.26.384.44 0 .877-.128 1.26-.383l7.24-4.83v10.022c0 .69-.56 1.25-1.25 1.25z"})
                            #dm出来るかどうかの確認
                            try:
                                #len(dm) >=2:dm可能 len(dm) <2:dm不可
                                if len(dm) >=2:
                                    url_list.append(url)
                                    follow_list.append(follow)
                                    follower_list.append(follower)
                                    dm_list.append("可能")
                                elif len(dm) <2:
                                    url_list.append(url)
                                    follow_list.append(follow)
                                    follower_list.append(follower)
                                    dm_list.append("不可")
                            #dmの確認ができなかったとき
                            except:
                                url_list.append(url)
                                follow_list.append(follow)
                                follower_list.append(follower)
                                dm_list.append("不明")
                        #鍵あり
                        elif len(key) >= 1:
                            text = soup.find_all("div",attrs={"class":"r-qvutc0","role":"button","tabindex":"0","dir":"auto"})
                            #follow
                            follow = text[1].get_text().replace("Following","")
                            #follower
                            follower = text[2].get_text().replace("Followers","")
                            #dm
                            dm = soup.find_all("path",attrs={"d":"M19.25 3.018H4.75C3.233 3.018 2 4.252 2 5.77v12.495c0 1.518 1.233 2.753 2.75 2.753h14.5c1.517 0 2.75-1.235 2.75-2.753V5.77c0-1.518-1.233-2.752-2.75-2.752zm-14.5 1.5h14.5c.69 0 1.25.56 1.25 1.25v.714l-8.05 5.367c-.273.18-.626.182-.9-.002L3.5 6.482v-.714c0-.69.56-1.25 1.25-1.25zm14.5 14.998H4.75c-.69 0-1.25-.56-1.25-1.25V8.24l7.24 4.83c.383.256.822.384 1.26.384.44 0 .877-.128 1.26-.383l7.24-4.83v10.022c0 .69-.56 1.25-1.25 1.25z"})
                            #dm出来るかどうかの確認
                            try:
                                #len(dm) >=2:dm可能 len(dm) <2:dm不可
                                if len(dm) >=2:
                                    url_list.append(url)
                                    follow_list.append(follow)
                                    follower_list.append(follower)
                                    dm_list.append("可能")
                                elif len(dm) <2:
                                    url_list.append(url)
                                    follow_list.append(follow)
                                    follower_list.append(follower)
                                    dm_list.append("不可")
                            #dmの確認ができなかったとき
                            except:
                                url_list.append(url)
                                follow_list.append(follow)
                                follower_list.append(follower)
                                dm_list.append("不明")
                    #urlが無効のとき
                    except:
                        url_list.append(url)
                        follow_list.append("error")
                        follower_list.append("error")
                        dm_list.append("error")
                #urlを取れなかったときのループ
                except:
                    url_list.append(url)
                    follow_list.append("error")
                    follower_list.append("error")
                    dm_list.append("error")
                #urlが"error"のときの処理
            elif url=="error":
                url_list.append(url)
                follow_list.append("error")
                follower_list.append("error")
                dm_list.append("error")
            #途中で不都合が起きているときの処理
            if i+1  != len(follow_list):
                print("なんか間違ってるよ")
                print(i+1,len(follow_list))
            #滞りなく処理が終わったときの処理
            else:
                print(i+1,follower_list[-1])
        print(len(follow_list),",",follow_list)
        driver.quit()
        return url_list,follow_list,follower_list,dm_list
    '''
    accound_id:accountが格納されたlist , account_pass:passwordが格納されたlist , start_num:開始したい番号 , 
    df:実行したいurlのは言ったDataframe(100,-1)の形状の必要がある ,inctance = GetTwitter(home_url)
    '''
    def get_loop(self,account_id,account_pass,start_num,df,instance):
        print("start1")
        instance =  instance
        if start_num == 0:
            start =0
        else:
            start = (start_num-1)/100  #101番目のようになるため(100ごと)
    #     アカウントの個数だけloopをさせる
        print("start1")
        for i in range(len(account_id)):
            tw_id = account_id[i]
            password = account_pass[i]
            dfdf =df[start+i]
    #         空のリストをセット(ここにid島の情報が入り最終的にpd.concatでまとめられる)
            a=[]
            b=[]
            c=[]
            d=[]
            #get_tweet関数を行う。
            #get_tweetの途中でエラーが起きたら今までの分をreturnする
            try:
                a,b,c,d=instance.get_tweet(dfdf,tw_id,password,instance,url_list=[],follow_list=[],follower_list=[],dm_list=[])
            #get_tweetの途中でエラーが起きた行に関しては
            except:
                #100この要素を持つ空のリストを作成
                lst = ["" for i in range(100)]
                #何もできなかった行に関してはからのリストを追加する
                a,b,c,d= lst,lst,lst,lst
                self.set_error.append("{}:番号は{}".format(tw_id,str(start_num*i)))

        #     得られたid等を統合する。abcはループのたびに殻にされるのでリセットされる
            self.url_wrapper.append(a)
            self.follow_wrapper.append(b)
            self.follower_wrapper.append(c)
            self.dm_wrapper.append(d)
            print(self.url_wrapper)
            print(self.follow_wrapper)
            print(self.follower_wrapper)
            print(self.dm_wrapper)
            print(self.set_error)
    #     得られたデータをpandasの形式に変更
        li1 = pd.DataFrame(self.url_wrapper)    
        li2 = pd.DataFrame(self.follow_wrapper)
        li3 = pd.DataFrame(self.follower_wrapper)
        li4 = pd.DataFrame(self.dm_wrapper)
        print(li1.T)
        print(li2.T)
        print(li3.T)
        print(li4.T)
    #     カラム名をセット
        col = ["twitter_id","follow数","follower数","dm可否"]
    #     それぞれの情報を列方向にまとめる
        twtw = pd.concat([li1.T,li2.T,li3.T,li4.T],axis =1)
        twtw = twtw.iloc[:,[0,len(account_id),len(account_id)*2,len(account_id)*3]]
        twtw.columns = col
        return twtw,li1,li2,li3,li4

        
 

In [45]:
home_url  = "https://twitter.com/home"

In [46]:
len(account_id)

10

In [47]:
def main():
    twitter = GetTwitter()
    twtw,li1,li2,li3,li4=twitter.get_loop(account_id , account_pass,0,dfdf1,twitter)
    return twtw,li1,li2,li3,li4

In [48]:
if __name__ == "__main__":
    a,b,c,d,e=main()

start1
start1


<ipython-input-44-ff87a78ce684>:31: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)


[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]
[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]
[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 

<ipython-input-44-ff87a78ce684>:31: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)


[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]
[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '

...

[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

<ipython-input-44-ff87a78ce684>:31: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)
<ipython-input-44-ff87a78ce684>:31: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)


[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

<ipython-input-44-ff87a78ce684>:31: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)
<ipython-input-44-ff87a78ce684>:31: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)


[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

In [90]:
dfdf2

,0
0,https://twitter.com/kakerukaeru
1,https://twitter.com/nipoko
2,https://twitter.com/cureseven
3,https://twitter.com/satorumpen
4,https://twitter.com/Otokomae
...,...
95,https://twitter.com/fantatchi
96,https://twitter.com/tyokomine
97,https://twitter.com/yuu_nkjm
98,https://twitter.com/doraTeX


In [91]:
twitter = GetTwitter()

In [92]:
a7,b7,c7,d7,e7=twitter.get_loop(account_id_test , account_pass_test,0,dfdf2,twitter)

start1
start1


<ipython-input-85-ff87a78ce684>:31: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)


100 回ループを起こします
1 518 
2 269 
3 2,178 
4 305 
5 1,133 
6 201 
7 323 
8 1,356 
9 90 
10 1,389 
11 213 
12 error
13 180 
14 3,045 
15 1,779 
16 24 
17 6 
18 92 
19 8,106 
20 369 
21 6,037 
22 297 
23 312 
24 543 
25 4,720 
26 error
27 2,020 
28 16 
29 17 
30 309 
31 299 
32 error
33 435 
34 696 
35 error
36 21 
37 error
38 error
39 599 
40 327 
41 224 
42 307 
43 2,484 
44 419 
45 1 Follower
46 968 
47 4,559 
48 280 
49 33 
50 375 
51 1,134 
52 904 
53 1 Follower
54 75 
55 1,385 
56 197 
57 1,095 
58 1,049 
59 705 
60 4,976 
61 1,740 
62 6 
63 147 
64 3 
65 307 
66 2,342 
67 752 
68 1,058 
69 error
70 25.3K 
71 539 
72 376 
73 351 
74 811 
75 283 
76 3 
77 222 
78 0 
79 811 
80 3,113 
81 298 
82 167 
83 616 
84 error
85 32 
86 58 
87 414 
88 173 
89 error
90 63 
91 315 
92 1,301 
93 error
94 389 
95 201 
96 286 
97 322 
98 397 
99 1,994 
100 2 
100 , ['856 ', '420 ', '1,500 ', '546 ', '1,735 ', '334 ', '72 ', '91 ', '116 ', '802 ', '327 ', 'error', '374 ', '2,096 ', '4,960 ', '84 ', '28 '

In [93]:
#出てくるデータの統合
def melt(a,b,c,d):  
    bb = pd.melt(a.T).iloc[:,1:]
    cc = pd.melt(b.T).iloc[:,1:]
    dd = pd.melt(c.T).iloc[:,1:]
    ee = pd.melt(d.T).iloc[:,1:]
    df = pd.concat([bb,cc,dd,ee],axis=1)
    return df

In [94]:
dfff = melt(b7,c7,d7,e7)

In [95]:
dfff.isna().sum()

value    0
value    0
value    0
value    0
dtype: int64

In [96]:
k = dfff.dropna()

In [97]:
k.columns = col

NameError: name 'col' is not defined

In [99]:
k.columns = ["twitterID","フォロー","フォロワー","DM可否"]

In [101]:
k.twitterID-"https://twitter.com/"

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [102]:
k.to_excel("訂正版1_12.xlsx")

In [57]:
ls

3001_7000.xlsx                  note_twitter5000_6000.csv
3001_7000_kai.xlsx              note_twitter6001_.csv
Untitled.ipynb                  note_twitter7001_.csv
aaa/                            note_twitterdata0-1000.csv
complete_notedata.csv           note_twitter用.ipynb
complete_notedata.xlsx          note実験用.ipynb
dm.png                          noteのユーザー.xlsx
kai.csv                         twitter_note_user1001-6000.csv
note.ipynb                      twitter_url.csv
note_list1_3000.csv             twitter_user-by-note.ipynb
note_list7001_8229.csv          user_note
note_twitter0_4999.csv          user_note.csv


In [158]:
b5.duplicated()

0     False
1     False
2     False
3      True
4      True
      ...  
68    False
69    False
70    False
71    False
72    False
Length: 73, dtype: bool

In [156]:
b5iloc[:,40:]

,40,41,42,43,44,45,46,47,48,49,...,63,64,65,66,67,68,69,70,71,72
0,error,,,error,error,error,error,error,error,error,...,error,error,https://twitter.com/boo_booko,error,error,error,https://twitter.com/Shalalanet_web,https://twitter.com/daisukesasaki56,https://twitter.com/Steave_Hiro18,error
1,error,,,error,https://twitter.com/gopherkawaii,error,https://twitter.com/kawac2236,https://twitter.com/nanakenashi,https://twitter.com/0Q73yowjRw0hVSP,https://twitter.com/dotou_eng,...,https://twitter.com/chikataga,https://twitter.com/dayblog510,error,https://twitter.com/1496yuta1,https://twitter.com/kikuyuribotan,error,error,error,error,https://twitter.com/RYO_hagicf
2,https://twitter.com/chaosm3h,,,https://twitter.com/chaosm3h,https://twitter.com/RSyuki_takayama,https://twitter.com/yuuhei0523,error,https://twitter.com/MomoNashiLychee,error,https://twitter.com/AKiRRRA19,...,https://twitter.com/YouManagements,https://twitter.com/Romimi__2020,error,https://twitter.com/F_SHOGO27,https://twitter.com/thorikawa,https://twitter.com/guitar_blog_Yu,https://twitter.com/off2white,error,error,error
3,error,,,error,error,https://twitter.com/yu_mashirou,error,https://twitter.com/qnipa928,error,https://twitter.com/somtos14,...,error,https://twitter.com/Triaezu1,https://twitter.com/akihiromorinaka,error,error,error,error,error,https://twitter.com/raba0224,error
4,https://twitter.com/SkillupY,,,https://twitter.com/SkillupY,https://twitter.com/shotaokb,https://twitter.com/Higashidadan,error,error,https://twitter.com/gt_release,https://twitter.com/kazystkd,...,error,https://twitter.com/quiqui59911225,https://twitter.com/ee_arts2020,error,error,error,https://twitter.com/mshr_ktmr,error,https://twitter.com/pujisi,https://twitter.com/nakahara_dan_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,error,,,error,error,https://twitter.com/program_salons,https://twitter.com/19_8_14_7_,error,error,error,...,error,https://twitter.com/DrudruSh,https://twitter.com/kamitonendo,https://twitter.com/hibikaze2,error,error,https://twitter.com/lmlzoqIVkPK8ZAf,https://twitter.com/hayashi_web_en,https://twitter.com/kodomoKazu,https://twitter.com/kojiro_ozaki
96,https://twitter.com/kbk_RK,,,https://twitter.com/kbk_RK,error,https://twitter.com/Yuhki_840,https://twitter.com/i_my_318,https://twitter.com/Pazuuu_UA,https://twitter.com/fumi_042,error,...,https://twitter.com/hobitsevenstar,https://twitter.com/umuscle_channel,https://twitter.com/KKTWSW2vGelPFUV,https://twitter.com/scraping,error,error,error,error,error,https://twitter.com/ERO_engineer
97,https://twitter.com/hagure_engineer,,,https://twitter.com/hagure_engineer,error,https://twitter.com/DenkiShirai,error,https://twitter.com/tagaranoko1014,https://twitter.com/AkiGenialTech1,error,...,error,https://twitter.com/hotz_91,https://twitter.com/wospii,https://twitter.com/SR0t1rVGQZ4nyfB,error,https://twitter.com/takahanda,https://twitter.com/nakamaru121,error,https://twitter.com/mrmrmrn_p_c,error
98,https://twitter.com/1026NT,,,https://twitter.com/1026NT,error,https://twitter.com/actyhh3,error,https://twitter.com/n1kaera,error,error,...,error,https://twitter.com/seosppin903,https://twitter.com/sk48801728,https://twitter.com/hide3995,https://twitter.com/garamon306,https://twitter.com/melumuu_,error,error,error,https://twitter.com/wfkw2kHlrgvsFZ4


In [248]:
dfdfdf1 = b5.iloc[20:,:].drop([40,41,42])
dfdfdf2 = c5.iloc[20:,:].drop([40,41,42])
dfdfdf3 = d5.iloc[20:,:].drop([40,41,42])
dfdfdf4 = e5.iloc[20:,:].drop([40,41,42])

In [249]:
data = melt(dfdfdf1,dfdfdf2,dfdfdf3,dfdfdf4)

In [252]:
data.columns=col

In [265]:
data.to_csv("twitter_note_user1001-6000.csv")

In [266]:
ls

3001_7000.xlsx                  note_twitter6001_.csv
3001_7000_kai.xlsx              note_twitter7001_.csv
Untitled.ipynb                  note_twitterdata0-1000.csv
complete_notedata.csv           note_twitter用.ipynb
complete_notedata.xlsx          note実験用.ipynb
dm.png                          noteのユーザー.xlsx
kai.csv                         twitter_note_user1001-6000.csv
note.ipynb                      twitter_url.csv
note_list1_3000.csv             twitter_user-by-note.ipynb
note_list7001_8229.csv          user_note
note_twitter0_4999.csv          user_note.csv
note_twitter5000_6000.csv


In [264]:
data.iloc[1000:,:]

,url,フォロー,フォロワー,dm可否
1000,https://twitter.com/kazoo_tnk,17,1 Follower,不可
1001,https://twitter.com/shtnkgm,71,545,可能
1002,https://twitter.com/ikasumi_wt,"1,331","1,054",不可
1003,https://twitter.com/gorius_48,error,error,error
1004,https://twitter.com/K_Tiva,"1,066",847,不可
...,...,...,...,...
4995,https://twitter.com/kojiro_ozaki,7,4,不可
4996,https://twitter.com/ERO_engineer,559,398,可能
4997,error,error,error,error
4998,https://twitter.com/wfkw2kHlrgvsFZ4,0,0,不可


In [262]:
df[2001]

'https://twitter.com/shtnkgm'

In [191]:
bfin = b5[~b5.duplicated()].iloc[3:,]

In [202]:
nnn=bfin.reset_index().iloc[:,1:]

In [258]:
nnn

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,https://twitter.com/tatsuroro,https://twitter.com/takumi_funasaka,error,error,https://twitter.com/chamenma,https://twitter.com/rllllho,https://twitter.com/tomobop,error,error,error,...,error,https://twitter.com/katsuengineer,error,https://twitter.com/taigaa_t,https://twitter.com/CreatorQsF,error,error,https://twitter.com/isorofree,error,https://twitter.com/nabe_no_suke81
1,error,error,error,https://twitter.com/deg84,https://twitter.com/wslngmgn,https://twitter.com/maosec,error,https://twitter.com/_akira19,https://twitter.com/kury0r,https://twitter.com/kedev01,...,error,error,error,https://twitter.com/ClipRipple,error,https://twitter.com/taka0189469,https://twitter.com/massage117_,https://twitter.com/imasara_king,error,error
2,https://twitter.com/itachi__itachi,https://twitter.com/tofu762,https://twitter.com/sugarxfebry,https://twitter.com/_incolors,error,https://twitter.com/Yoppeenn,https://twitter.com/omotenashitaro,error,error,error,...,https://twitter.com/tatsuaki85,https://twitter.com/excitement_hiro,https://twitter.com/trading_xxx,https://twitter.com/masanarihori,error,https://twitter.com/katsuya71109608,https://twitter.com/rotomx,https://twitter.com/_leo_isaac,error,https://twitter.com/mannen0601
3,https://twitter.com/nikuq299com,https://twitter.com/ShimadaAtsuto,https://twitter.com/satotin,https://twitter.com/ysk1412,https://twitter.com/2chiyaM,https://twitter.com/ryamakuchi,https://twitter.com/buddist1234,https://twitter.com/manaty_blog,https://twitter.com/hrk_okd,https://twitter.com/fkmt_pei,...,https://twitter.com/beryu,https://twitter.com/yakamatsu2,https://twitter.com/sota1235,https://twitter.com/h_tsuruo,https://twitter.com/RyoKojima4,https://twitter.com/shuuuunchan31,https://twitter.com/tim_yone,error,error,https://twitter.com/_tamitom
4,error,error,error,https://twitter.com/nabe__kurage,https://twitter.com/ko_chan214,https://twitter.com/iwasiman,error,error,https://twitter.com/a7Xsc2XHeBTwHLm,https://twitter.com/lVlnaga,...,https://twitter.com/39rdtks,error,https://twitter.com/hayatobell,https://twitter.com/tetsuyasuetake,https://twitter.com/moekyuneom,https://twitter.com/chantama144,https://twitter.com/Baseballyama,https://twitter.com/daigaloo,error,error
5,error,error,https://twitter.com/_munazo_,https://twitter.com/tumuji_D,https://twitter.com/kyo__hei,error,error,https://twitter.com/JotarO_Oyanagi,error,https://twitter.com/HKK_note,...,https://twitter.com/yossy_0021,error,error,https://twitter.com/smdragunov17,https://twitter.com/nbc_gt,https://twitter.com/hiro_em3,https://twitter.com/NaochaelJ,https://twitter.com/buebue4632,error,https://twitter.com/katsuki_wataru
6,https://twitter.com/kenken1057,https://twitter.com/totti_as_10,https://twitter.com/r_karotou,https://twitter.com/utapioka,https://twitter.com/akimaru_samurai,error,https://twitter.com/emegane,error,https://twitter.com/HimaIse,https://twitter.com/o_kubos20,...,https://twitter.com/ya6maDev,https://twitter.com/itomakiweb,https://twitter.com/nekn3x,https://twitter.com/kaori_ouchiit,https://twitter.com/storm_btc,error,https://twitter.com/KaySquare1231,error,error,https://twitter.com/kyam28
7,error,error,error,error,https://twitter.com/jacknightmare13,error,https://twitter.com/SugimoriKatsu1,https://twitter.com/DMswAr,https://twitter.com/kanameg,https://twitter.com/peromasamune,...,error,error,https://twitter.com/Tsutomu_eng,https://twitter.com/mashi__Co,https://twitter.com/yuyudora622,https://twitter.com/moroku0519,https://twitter.com/akisora001,error,https://twitter.com/BlackFlagSVG,https://twitter.com/rinrinxyz7
8,https://twitter.com/kota_nomura_,https://twitter.com/tantakan_,https://twitter.com/awsmgs,https://twitter.com/kumamoto_kengo,https://twitter.com/yuuki_mine,error,https://twitter.com/u_uusu_,https://twitter.com/mmrakt0716,error,https://twitter.com/nyanyas222,...,https://twitter.com/kng580,https://twitter.com/tomk79,error,error,https://twitter.com/markeOutput,error,https://twitter.com

In [209]:
df[2900:]

2900        https://twitter.com/khanom_pan_
2901       https://twitter.com/pavement1234
2902          https://twitter.com/yeyshonan
2903                                  error
2904      https://twitter.com/ystk_namekawa
                       ...                 
8224          https://twitter.com/CalifDrea
8225           https://twitter.com/sa178kla
8226    https://twitter.com/o4rKjqaANelludp
8227             https://twitter.com/makies
8228        https://twitter.com/rascal_0304
Name: twitter_url, Length: 5329, dtype: object

In [186]:
df[df=="https://twitter.com/laphroaig_PPC"]

900    https://twitter.com/laphroaig_PPC
Name: twitter_url, dtype: object

In [241]:
# 40,41,42が無駄な行
df2000 = melt(b1[40:,].drop([40,41,42]))

KeyError: (slice(40, None, None),)

In [101]:
ddd = melt(b,c,d,e)

In [112]:
df[995:1000]

995    error
996    error
997    error
998    error
999    error
Name: twitter_url, dtype: object

In [129]:
ddd.iloc[300:,]

,value,value,value,value
300,https://twitter.com/ino_be_ta,37,37,不可
301,https://twitter.com/pt_sasakawa,67,"5,855",可能
302,https://twitter.com/CpYoMXLnSo,406,"1,040",可能
303,https://twitter.com/MK__gorilla,867,"1,111",可能
304,https://twitter.com/yosuke_ushio),error,error,error
...,...,...,...,...
995,error,error,error,error
996,error,error,error,error
997,error,error,error,error
998,error,error,error,error


In [128]:
df[300:]

300           https://twitter.com/ino_be_ta
301         https://twitter.com/pt_sasakawa
302          https://twitter.com/CpYoMXLnSo
303         https://twitter.com/MK__gorilla
304       https://twitter.com/yosuke_ushio)
                       ...                 
8224          https://twitter.com/CalifDrea
8225           https://twitter.com/sa178kla
8226    https://twitter.com/o4rKjqaANelludp
8227             https://twitter.com/makies
8228        https://twitter.com/rascal_0304
Name: twitter_url, Length: 7929, dtype: object

In [118]:
df

0               https://twitter.com/issei_y
1                https://twitter.com/konpyu
2       https://twitter.com/shiraishikatsuy
3                https://twitter.com/shu223
4             https://twitter.com/snakajima
                       ...                 
8224          https://twitter.com/CalifDrea
8225           https://twitter.com/sa178kla
8226    https://twitter.com/o4rKjqaANelludp
8227             https://twitter.com/makies
8228        https://twitter.com/rascal_0304
Name: twitter_url, Length: 8229, dtype: object

In [39]:
for i in range(len(account_id)):
    twitter.login(account_id[i],account_pass[i])

<ipython-input-36-45a5f75c68b4>:29: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)
<ipython-input-36-45a5f75c68b4>:29: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)
<ipython-input-36-45a5f75c68b4>:29: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)
<ipython-input-36-45a5f75c68b4>:29: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)
<ipython-input-36-45a5f75c68b4>:29: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)
<ipython-input-36-45a5f75c68b4>:29: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=self.options)
<ipython-input-36-45a5f75c68b4>:29: DeprecationWarning: use options instead of chrome_options


In [52]:
j=np.arange(0,100)

In [218]:
#b1エラーなし
b1.iloc[20:,:]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
20,error,https://twitter.com/zoomin0123,https://twitter.com/cocodrips,https://twitter.com/to_ryo_endo,https://twitter.com/shajiku_works,error,https://twitter.com/naruktrg,error,https://twitter.com/tsukahika083131,error,...,https://twitter.com/TomHack2020,https://twitter.com/rolling_amaebi,error,error,https://twitter.com/SUPERDONUT9,https://twitter.com/ku__ma59,https://twitter.com/ppxFP3xjNIEd8RJ,https://twitter.com/super_gel,error,https://twitter.com/on_bias
21,https://twitter.com/tatsuroro,https://twitter.com/takumi_funasaka,error,error,https://twitter.com/chamenma,https://twitter.com/rllllho,https://twitter.com/tomobop,error,error,error,...,error,https://twitter.com/katsuengineer,error,https://twitter.com/taigaa_t,https://twitter.com/CreatorQsF,error,error,https://twitter.com/isorofree,error,https://twitter.com/nabe_no_suke81
22,error,error,error,https://twitter.com/deg84,https://twitter.com/wslngmgn,https://twitter.com/maosec,error,https://twitter.com/_akira19,https://twitter.com/kury0r,https://twitter.com/kedev01,...,error,error,error,https://twitter.com/ClipRipple,error,https://twitter.com/taka0189469,https://twitter.com/massage117_,https://twitter.com/imasara_king,error,error
23,https://twitter.com/itachi__itachi,https://twitter.com/tofu762,https://twitter.com/sugarxfebry,https://twitter.com/_incolors,error,https://twitter.com/Yoppeenn,https://twitter.com/omotenashitaro,error,error,error,...,https://twitter.com/tatsuaki85,https://twitter.com/excitement_hiro,https://twitter.com/trading_xxx,https://twitter.com/masanarihori,error,https://twitter.com/katsuya71109608,https://twitter.com/rotomx,https://twitter.com/_leo_isaac,error,https://twitter.com/mannen0601
24,https://twitter.com/nikuq299com,https://twitter.com/ShimadaAtsuto,https://twitter.com/satotin,https://twitter.com/ysk1412,https://twitter.com/2chiyaM,https://twitter.com/ryamakuchi,https://twitter.com/buddist1234,https://twitter.com/manaty_blog,https://twitter.com/hrk_okd,https://twitter.com/fkmt_pei,...,https://twitter.com/beryu,https://twitter.com/yakamatsu2,https://twitter.com/sota1235,https://twitter.com/h_tsuruo,https://twitter.com/RyoKojima4,https://twitter.com/shuuuunchan31,https://twitter.com/tim_yone,error,error,https://twitter.com/_tamitom
25,error,error,error,https://twitter.com/nabe__kurage,https://twitter.com/ko_chan214,https://twitter.com/iwasiman,error,error,https://twitter.com/a7Xsc2XHeBTwHLm,https://twitter.com/lVlnaga,...,https://twitter.com/39rdtks,error,https://twitter.com/hayatobell,https://twitter.com/tetsuyasuetake,https://twitter.com/moekyuneom,https://twitter.com/chantama144,https://twitter.com/Baseballyama,https://twitter.com/daigaloo,error,error
26,error,error,https://twitter.com/_munazo_,https://twitter.com/tumuji_D,https://twitter.com/kyo__hei,error,error,https://twitter.com/JotarO_Oyanagi,error,https://twitter.com/HKK_note,...,https://twitter.com/yossy_0021,error,error,https://twitter.com/smdragunov17,https://twitter.com/nbc_gt,https://twitter.com/hiro_em3,https://twitter.com/NaochaelJ,https://twitter.com/buebue4632,error,https://twitter.com/katsuki_wataru
27,https://twitter.com/kenken1057,https://twitter.com/totti_as_10,https://twitter.com/r_karotou,https://twitter.com/utapioka,https://twitter.com/akimaru_samurai,error,https://twitter.com/emegane,error,https://twitter.com/HimaIse,https://twitter.com/o_kubos20,...,https://twitter.com/ya6maDev,https://twitter.com/itomakiweb,https://twitter.com/nekn3x,https://twitter.com/kaori_ouchiit,https://twitter.com/storm_btc,error,https://twitter.com/KaySquare1231,error,error,https://twitter.com/kyam28
28,error,error,error,error,https://twitter.com/jacknightmare13,error,https://twitter.com/SugimoriKatsu1,https://twitter.com/DMswAr,https://twitter.com/kanameg,https://twitter.com/peromasamune,...,error,error,https://twitter.com/Tsutomu_eng,https://twitter.com/mashi__Co,https://twitter.com/yuyudora622,https://twitter.com/moroku0519,https://twitter.c

In [220]:
#b２エラーなし
b2.iloc[20:,:]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
20,error,https://twitter.com/zoomin0123,https://twitter.com/cocodrips,https://twitter.com/to_ryo_endo,https://twitter.com/shajiku_works,error,https://twitter.com/naruktrg,error,https://twitter.com/tsukahika083131,error,...,https://twitter.com/TomHack2020,https://twitter.com/rolling_amaebi,error,error,https://twitter.com/SUPERDONUT9,https://twitter.com/ku__ma59,https://twitter.com/ppxFP3xjNIEd8RJ,https://twitter.com/super_gel,error,https://twitter.com/on_bias
21,https://twitter.com/tatsuroro,https://twitter.com/takumi_funasaka,error,error,https://twitter.com/chamenma,https://twitter.com/rllllho,https://twitter.com/tomobop,error,error,error,...,error,https://twitter.com/katsuengineer,error,https://twitter.com/taigaa_t,https://twitter.com/CreatorQsF,error,error,https://twitter.com/isorofree,error,https://twitter.com/nabe_no_suke81
22,error,error,error,https://twitter.com/deg84,https://twitter.com/wslngmgn,https://twitter.com/maosec,error,https://twitter.com/_akira19,https://twitter.com/kury0r,https://twitter.com/kedev01,...,error,error,error,https://twitter.com/ClipRipple,error,https://twitter.com/taka0189469,https://twitter.com/massage117_,https://twitter.com/imasara_king,error,error
23,https://twitter.com/itachi__itachi,https://twitter.com/tofu762,https://twitter.com/sugarxfebry,https://twitter.com/_incolors,error,https://twitter.com/Yoppeenn,https://twitter.com/omotenashitaro,error,error,error,...,https://twitter.com/tatsuaki85,https://twitter.com/excitement_hiro,https://twitter.com/trading_xxx,https://twitter.com/masanarihori,error,https://twitter.com/katsuya71109608,https://twitter.com/rotomx,https://twitter.com/_leo_isaac,error,https://twitter.com/mannen0601
24,https://twitter.com/nikuq299com,https://twitter.com/ShimadaAtsuto,https://twitter.com/satotin,https://twitter.com/ysk1412,https://twitter.com/2chiyaM,https://twitter.com/ryamakuchi,https://twitter.com/buddist1234,https://twitter.com/manaty_blog,https://twitter.com/hrk_okd,https://twitter.com/fkmt_pei,...,https://twitter.com/beryu,https://twitter.com/yakamatsu2,https://twitter.com/sota1235,https://twitter.com/h_tsuruo,https://twitter.com/RyoKojima4,https://twitter.com/shuuuunchan31,https://twitter.com/tim_yone,error,error,https://twitter.com/_tamitom
25,error,error,error,https://twitter.com/nabe__kurage,https://twitter.com/ko_chan214,https://twitter.com/iwasiman,error,error,https://twitter.com/a7Xsc2XHeBTwHLm,https://twitter.com/lVlnaga,...,https://twitter.com/39rdtks,error,https://twitter.com/hayatobell,https://twitter.com/tetsuyasuetake,https://twitter.com/moekyuneom,https://twitter.com/chantama144,https://twitter.com/Baseballyama,https://twitter.com/daigaloo,error,error
26,error,error,https://twitter.com/_munazo_,https://twitter.com/tumuji_D,https://twitter.com/kyo__hei,error,error,https://twitter.com/JotarO_Oyanagi,error,https://twitter.com/HKK_note,...,https://twitter.com/yossy_0021,error,error,https://twitter.com/smdragunov17,https://twitter.com/nbc_gt,https://twitter.com/hiro_em3,https://twitter.com/NaochaelJ,https://twitter.com/buebue4632,error,https://twitter.com/katsuki_wataru
27,https://twitter.com/kenken1057,https://twitter.com/totti_as_10,https://twitter.com/r_karotou,https://twitter.com/utapioka,https://twitter.com/akimaru_samurai,error,https://twitter.com/emegane,error,https://twitter.com/HimaIse,https://twitter.com/o_kubos20,...,https://twitter.com/ya6maDev,https://twitter.com/itomakiweb,https://twitter.com/nekn3x,https://twitter.com/kaori_ouchiit,https://twitter.com/storm_btc,error,https://twitter.com/KaySquare1231,error,error,https://twitter.com/kyam28
28,error,error,error,error,https://twitter.com/jacknightmare13,error,https://twitter.com/SugimoriKatsu1,https://twitter.com/DMswAr,https://twitter.com/kanameg,https://twitter.com/peromasamune,...,error,error,https://twitter.com/Tsutomu_eng,https://twitter.com/mashi__Co,https://twitter.com/yuyudora622,https://twitter.com/moroku0519,https://twitter.c

In [230]:
#b3 40,41,42行目無駄

b3.iloc[40:,:].drop([40,41,42])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
43,error,error,https://twitter.com/chaosm3h,error,https://twitter.com/SkillupY,https://twitter.com/Jessicazu418,error,error,https://twitter.com/kaorioh_tech,https://twitter.com/SuzuBacks,...,error,https://twitter.com/daikon_oloshi,error,https://twitter.com/hirotaka_it,https://twitter.com/hagi60,error,https://twitter.com/kbk_RK,https://twitter.com/hagure_engineer,https://twitter.com/1026NT,https://twitter.com/k__t__t__
44,error,https://twitter.com/gopherkawaii,https://twitter.com/RSyuki_takayama,error,https://twitter.com/shotaokb,https://twitter.com/RAeMERqqFfj8zWb,https://twitter.com/idiot_radio_hy,error,https://twitter.com/WBboy32,error,...,error,https://twitter.com/Smith83040035,https://twitter.com/monika_masurao,error,error,error,error,error,error,https://twitter.com/haruo666
45,error,error,https://twitter.com/yuuhei0523,https://twitter.com/yu_mashirou,https://twitter.com/Higashidadan,error,error,https://twitter.com/tDKcJ5QVAcKFMDd,error,error,...,error,error,https://twitter.com/itjca30,https://twitter.com/etBeEP5e7dwmw7P,error,https://twitter.com/program_salons,https://twitter.com/Yuhki_840,https://twitter.com/DenkiShirai,https://twitter.com/actyhh3,error
46,error,https://twitter.com/kawac2236,error,error,error,error,https://twitter.com/amaduka,error,https://twitter.com/WLB25_DIARY,https://twitter.com/mygkrnk,...,error,error,https://twitter.com/yoshi25133,https://twitter.com/moimoi1220,https://twitter.com/hyamak,https://twitter.com/19_8_14_7_,https://twitter.com/i_my_318,error,error,error
47,error,https://twitter.com/nanakenashi,https://twitter.com/MomoNashiLychee,https://twitter.com/qnipa928,error,error,https://twitter.com/finengine,https://twitter.com/kenichi_ot,error,https://twitter.com/applestarsenal,...,error,error,error,error,https://twitter.com/koujienami,error,https://twitter.com/Pazuuu_UA,https://twitter.com/tagaranoko1014,https://twitter.com/n1kaera,https://twitter.com/mico_engineer
48,error,https://twitter.com/0Q73yowjRw0hVSP,error,error,https://twitter.com/gt_release,https://twitter.com/skanno,https://twitter.com/anna_mof_pr,https://twitter.com/u_jin,error,https://twitter.com/yotuba_eng,...,error,https://twitter.com/akira3115,https://twitter.com/SitommyBlog,https://twitter.com/yushindiabolo,error,error,https://twitter.com/fumi_042,https://twitter.com/AkiGenialTech1,error,error
49,error,https://twitter.com/dotou_eng,https://twitter.com/AKiRRRA19,https://twitter.com/somtos14,https://twitter.com/kazystkd,https://twitter.com/loveeeeeeeeni,https://twitter.com/katsumura1123,error,error,https://twitter.com/mizuki_r,...,https://twitter.com/AFpiQKy7qHKllMh,error,error,https://twitter.com/kazu_kingchan,https://twitter.com/IBKblue_cha,error,error,error,error,https://twitter.com/OKAKI_SE
50,error,error,error,https://twitter.com/takuma_m4p,https://twitter.com/web72915350,error,error,https://twitter.com/undersooon,https://twitter.com/whiteeel_it,error,...,https://twitter.com/zundan,error,https://twitter.com/Nigishi3,error,error,https://twitter.com/George60711064,error,error,https://twitter.com/miy_yan,error
51,https://twitter.com/bc_yuuuuuki,error,https://twitter.com/KAGUYA_HOE,https://twitter.com/ozonosho,https://twitter.com/sachi164cm222,error,https://twitter.com/mitsui_mk,https://twitter.com/timespacecreate,error,https://twitter.com/kouhei_o_o,...,https://twitter.com/yagiyuki06,https://twitter.com/a_makoto33,https://twitter.com/nao0,error,https://twitter.com/senaianes,error,error,error,error,error
52,error,error,error,error,https://twitter.com/miitame,error,https://twitter.com/Gamiblogger,error,https://twitter.com/RYamatech,error,...,https://twitter.com/masP047,https://twitter.com/Wataru_Gucci,https://twitter.com/seijuro1110,https://twitter.com/tuttytutty5432,error,https://twitter.com/supra_dee,https://twitter.com/h_hoshikawa,error,https://twitter.com/G28344302,error


In [237]:
#b4 40,41,42行目無駄

b4.iloc[53:,:]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
53,error,error,error,https://twitter.com/hanage_rice,https://twitter.com/lia6112,https://twitter.com/anchaaaan7,https://twitter.com/R_Az_Tech,error,error,https://twitter.com/mosurisu,...,https://twitter.com/pinkcosmos47,https://twitter.com/halu834,https://twitter.com/QAengineer14,https://twitter.com/yuuya08110,https://twitter.com/i_nobo,error,error,error,https://twitter.com/rinnkisi,error
54,https://twitter.com/DoReMiHwaDonatu,https://twitter.com/note36542087,https://twitter.com/enkey_qt,error,https://twitter.com/shohei52082238,https://twitter.com/howtomake_idea,https://twitter.com/TatsuyaR7,https://twitter.com/hirosuke_ja,error,error,...,error,error,https://twitter.com/yyamaguchi0611?s=09,https://twitter.com/sirokuma_diary,https://twitter.com/takakunn0516,error,https://twitter.com/a1_shun,https://twitter.com/koheiarai94,https://twitter.com/acquamusica,https://twitter.com/rem6134
55,https://twitter.com/vandammelab,error,https://twitter.com/NonotameCh,error,https://twitter.com/tworks55,https://twitter.com/nakoche24,https://twitter.com/umisukinaacha,https://twitter.com/oga_4528,error,https://twitter.com/leon_pdm,...,https://twitter.com/hisui9jade,https://twitter.com/ydaTakahiroSeki,https://twitter.com/en8fc,error,https://twitter.com/piyozo2,error,error,error,error,error
56,https://twitter.com/bravengineer,https://twitter.com/osakana1028,https://twitter.com/lH6kCZLqRWmdv3o,error,https://twitter.com/judo5001,error,https://twitter.com/bloomtherose,error,https://twitter.com/hiroki_ddd,https://twitter.com/thunderdoll3,...,error,https://twitter.com/nopakat2525,error,error,https://twitter.com/FunnnnLifeeee,error,https://twitter.com/fan_pg_ah,https://twitter.com/tubdaka616,https://twitter.com/kachinige_com,https://twitter.com/Nidhog0806
57,https://twitter.com/yaaamaaaguuu,https://twitter.com/naokichi2020,https://twitter.com/6ogan,error,https://twitter.com/Est_42,https://twitter.com/NoasaVn,https://twitter.com/masa4nn,error,https://twitter.com/milkmanscc,https://twitter.com/MotokiChina,...,https://twitter.com/gisi_outputit,error,error,https://twitter.com/FISHERS_studios,error,https://twitter.com/atsushi_pg,https://twitter.com/lazybouz,error,error,error
58,https://twitter.com/minimaristt,https://twitter.com/RPALTy_D,error,https://twitter.com/M_Tattin,error,error,https://twitter.com/yamatotakeru422,https://twitter.com/takabsd,error,error,...,error,https://twitter.com/doiken_,error,error,error,error,https://twitter.com/leonewfie,error,https://twitter.com/endo8og,https://twitter.com/JidaiTrans
59,https://twitter.com/million_momoban,https://twitter.com/dymori_z,https://twitter.com/yuukids,https://twitter.com/inventory_apim,https://twitter.com/A_MxT_,error,https://twitter.com/MasatoshiTakad2,error,https://twitter.com/appngmy,error,...,error,https://twitter.com/wtrkurihara,https://twitter.com/_yuzu_tmp,https://twitter.com/tene_singing,error,https://twitter.com/shakurein,error,https://twitter.com/free_motion,https://twitter.com/razukabie,error
60,error,error,error,error,https://twitter.com/YoshiFujisawa,error,error,error,https://twitter.com/konaka_coen,error,...,https://twitter.com/watanabe_sanda,https://twitter.com/eielh,https://twitter.com/n0442,error,error,error,https://twitter.com/Kzak1121,error,https://twitter.com/mirkokai,error
61,https://twitter.com/tobi462,error,https://twitter.com/yyyu0913,https://twitter.com/admin0907,https://twitter.com/KaoriNagoya_,https://twitter.com/chii_sun3,https://twitter.com/Actionfighter1,https://twitter.com/ssclimb1,https://twitter.com/yager,error,...,error,error,error,https://twitter.com/heyhey23478366,https://twitter.com/y0un0_,error,error,error,https://twitter.com/kinchan1110,error
62,error,https://twitter.com/katsusilva21,error,error,https://twitter.com/yasuo_teacher,https://twitter.com/JunShink,https://twitter.com/HR_man_yu,https://twitter.com/kon_yu,https://twitter.com/jun__tech,https://twitter.com/_fujitam,...,error,https://twitter.com/aranta_sre,h

In [239]:
df[4000:]

4000                                  error
4001                                  error
4002                                  error
4003        https://twitter.com/hanage_rice
4004            https://twitter.com/lia6112
                       ...                 
8224          https://twitter.com/CalifDrea
8225           https://twitter.com/sa178kla
8226    https://twitter.com/o4rKjqaANelludp
8227             https://twitter.com/makies
8228        https://twitter.com/rascal_0304
Name: twitter_url, Length: 4229, dtype: object

In [53]:
k.append(j)

In [55]:
kk=pd.DataFrame(k)

In [56]:
kk

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99


In [58]:
lst = ["" for i in range(100)]

In [65]:
start_num=101
i=2
tw_id="jjj"

In [77]:
lst

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'jjj:番号は202']

In [76]:
lst.append("{}:番号は{}".format(tw_id,str(start_num*i)))

In [74]:
"{}:番号は{}".format(tw_id,str((start_num-1)*i))

'jjj:番号は200'

In [132]:
ddd.to_csv("note_twitterdata0-1000.csv")

In [59]:
col=["url","フォロー","フォロワー","dm可否"]

In [246]:
ddd

,url,フォロー,フォロワー,dm可否
0,https://twitter.com/issei_y,751,27K,可能
1,https://twitter.com/konpyu,820,"2,054",可能
2,https://twitter.com/shiraishikatsuy,"5,001","4,084",可能
3,https://twitter.com/shu223,950,10.2K,可能
4,https://twitter.com/snakajima,300,20.7K,不可
...,...,...,...,...
995,error,error,error,error
996,error,error,error,error
997,error,error,error,error
998,error,error,error,error


In [247]:
ddd.to_csv("note_twitterdata0-1000.csv")

In [1]:
def login(tw_id,password):
        driver = webdriver.Chrome(chrome_options=.options)
        driver.get(home_url)
        time.sleep(5)
        
        # ユーザーidのinput要素を取得する
        input_name=  driver.find_element_by_name("session[username_or_email]")
        input_name.send_keys(tw_id)

        #  パスワードのinput要素を取得する
        input_pass = driver.find_element_by_name("session[password]")
        input_pass.send_keys(password)

        #  ログインボタンの要素を取得する
        login_button = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[2]/form/div/div[3]/div/div')
        login_button.click()

        time.sleep(1)
        return driver

SyntaxError: invalid syntax (<ipython-input-1-3720cb2725ab>, line 2)